In [1]:
import pandas as pd
#import sweetviz as sv
import os
import plotly.express as px
import seaborn as sns
import matplotlib as mlp
import psycopg2
import plotly.offline as pyo
import random
from dash import Dash, html, dcc, Input, Output
import dash_daq as daq
from flask import Flask
from gevent.pywsgi import WSGIServer

import os, signal

In [2]:
def connection(sql):
    con = psycopg2.connect(host='35.247.202.234', database='dw',
    user='consulta', password='teste123')
    cur = con.cursor()    
    cur.execute(sql)
    data = cur.fetchall()
    cols = []
    for elt in cur.description:
        cols.append(elt[0])

    df = pd.DataFrame(data=data,columns=cols)
    con.close();
    return df

In [3]:
query = """select * from public.fatores_est fe
inner join escolas.aluno a 
on fe.id_aluno = a.id_aluno 
inner join escolas.turma t 
on a.id_turma = t.id_turma 
inner join escolas.escola e 
on t.id_escola = e.cod_escola"""
df = connection(query)


In [4]:
df = df.loc[:,~df.columns.duplicated()].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 60 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   id                          62 non-null     int64              
 1   id_aluno                    62 non-null     int64              
 2   E_ESC1V                     62 non-null     float64            
 3   E_ESC2V                     62 non-null     float64            
 4   E_PROF1V                    62 non-null     float64            
 5   E_PROF2V                    62 non-null     float64            
 6   E_FAM1V                     62 non-null     float64            
 7   E_FAM2V                     62 non-null     float64            
 8   E_COM1V                     62 non-null     float64            
 9   E_COM2V                     62 non-null     float64            
 10  E_COM3V                     62 non-null     float64            


In [5]:
df.columns

Index(['id', 'id_aluno', 'E_ESC1V', 'E_ESC2V', 'E_PROF1V', 'E_PROF2V',
       'E_FAM1V', 'E_FAM2V', 'E_COM1V', 'E_COM2V', 'E_COM3V', 'E_EST1V',
       'E_EST2V', 'E_EST3V', 'E_ESC1C', 'E_ESC2C', 'E_PROF1C', 'E_PROF2C',
       'E_FAM1C', 'E_FAM2C', 'E_COM1C', 'E_COM2C', 'E_COM3C', 'E_EST1C',
       'E_EST2C', 'E_EST3C', 'created_date', 'updated_date', 'id_turma',
       'cod_matricula', 'nome_turma', 'nome_aluno', 'tp_sexo', 'id_escola',
       'modo_ensino', 'etapa_ensino', 'ano_turma', 'data_criacao',
       'resticao_de_atendimento', 'escola', 'cod_escola', 'uf', 'municipio',
       'localizacao', 'localidade_diferenciada', 'categoria_administrativa',
       'endereco', 'telefone', 'dependencia_administrativa',
       'categoria_escola', 'conveniada_poder_publico',
       'regulamentacao_conselho', 'porte', 'etapas_e_modalidade_ensino',
       'outras_ofertas', 'cod_regiao', 'micro', 'meso', 'regiao',
       'cod_estado'],
      dtype='object')

In [6]:
cross_tab_prop = df[['id_aluno','E_ESC1V', 'E_ESC2V', 'E_PROF1V', 'E_PROF2V',
       'E_FAM1V', 'E_FAM2V', 'E_COM1V', 'E_COM2V', 'E_COM3V', 'E_EST1V',
       'E_EST2V', 'E_EST3V', 'E_ESC1C', 'E_ESC2C', 'E_PROF1C', 'E_PROF2C',
       'E_FAM1C', 'E_FAM2C', 'E_COM1C', 'E_COM2C', 'E_COM3C', 'E_EST1C',
       'E_EST2C', 'E_EST3C']]

cross_tab_prop.set_index('id_aluno')
cross_tab_prop

,id_aluno,E_ESC1V,E_ESC2V,E_PROF1V,E_PROF2V,E_FAM1V,E_FAM2V,E_COM1V,E_COM2V,E_COM3V,...,E_PROF1C,E_PROF2C,E_FAM1C,E_FAM2C,E_COM1C,E_COM2C,E_COM3C,E_EST1C,E_EST2C,E_EST3C
0,14799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,19175,1.000000,3.000000,1.000000,1.000000,3.000000,1.000000,1.0,1.000000,1.000000,...,1,1,1,1,1,1,1,2,1,1
2,19174,1.000000,4.000000,1.000000,1.000000,3.000000,4.000000,1.0,1.000000,3.666667,...,1,1,1,2,1,1,3,2,1,3
3,19152,3.000000,1.000000,2.000000,1.000000,2.666667,4.000000,1.0,2.000000,2.666667,...,1,1,1,2,1,1,2,2,1,3
4,19150,3.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1,1,1,1,1,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,19010,1.333333,3.333333,4.000000,2.000000,2.333333,6.000000,1.0,3.000000,4.333333,...,2,1,1,3,1,2,3,2,1,3
58,18999,1.000000,1.000000,4.000000,2.000000,3.666667,4.000000,1.0,2.000000,2.000000,...,2,1,2,2,1,1,1,2,2,1
59,18994,1.000000,1.000000,2.000000,2.666667,3.666667,5.000000,1.0,5.666667,3.000000,...,1,2,2,2,1,3,2,3,3,1
60,18993,2.333333,3.000000,1.333333,1.666667,2.000000,1.666667,3.0,1.333333,1.666667,...,1,1,1,1,2,1,1,2,1,3


In [7]:
cross_tab_prop

,id_aluno,E_ESC1V,E_ESC2V,E_PROF1V,E_PROF2V,E_FAM1V,E_FAM2V,E_COM1V,E_COM2V,E_COM3V,...,E_PROF1C,E_PROF2C,E_FAM1C,E_FAM2C,E_COM1C,E_COM2C,E_COM3C,E_EST1C,E_EST2C,E_EST3C
0,14799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,19175,1.000000,3.000000,1.000000,1.000000,3.000000,1.000000,1.0,1.000000,1.000000,...,1,1,1,1,1,1,1,2,1,1
2,19174,1.000000,4.000000,1.000000,1.000000,3.000000,4.000000,1.0,1.000000,3.666667,...,1,1,1,2,1,1,3,2,1,3
3,19152,3.000000,1.000000,2.000000,1.000000,2.666667,4.000000,1.0,2.000000,2.666667,...,1,1,1,2,1,1,2,2,1,3
4,19150,3.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1,1,1,1,1,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,19010,1.333333,3.333333,4.000000,2.000000,2.333333,6.000000,1.0,3.000000,4.333333,...,2,1,1,3,1,2,3,2,1,3
58,18999,1.000000,1.000000,4.000000,2.000000,3.666667,4.000000,1.0,2.000000,2.000000,...,2,1,2,2,1,1,1,2,2,1
59,18994,1.000000,1.000000,2.000000,2.666667,3.666667,5.000000,1.0,5.666667,3.000000,...,1,2,2,2,1,3,2,3,3,1
60,18993,2.333333,3.000000,1.333333,1.666667,2.000000,1.666667,3.0,1.333333,1.666667,...,1,1,1,1,2,1,1,2,1,3


In [8]:
df_col = cross_tab_prop.loc[:, cross_tab_prop.columns.str.startswith("E_") | cross_tab_prop.columns.str.startswith("id")]
df_col = df_col.loc[:, df_col.columns.str.endswith("V")| cross_tab_prop.columns.str.startswith("id")]
df_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_aluno  62 non-null     int64  
 1   E_ESC1V   62 non-null     float64
 2   E_ESC2V   62 non-null     float64
 3   E_PROF1V  62 non-null     float64
 4   E_PROF2V  62 non-null     float64
 5   E_FAM1V   62 non-null     float64
 6   E_FAM2V   62 non-null     float64
 7   E_COM1V   62 non-null     float64
 8   E_COM2V   62 non-null     float64
 9   E_COM3V   62 non-null     float64
 10  E_EST1V   62 non-null     float64
 11  E_EST2V   62 non-null     float64
 12  E_EST3V   62 non-null     float64
dtypes: float64(12), int64(1)
memory usage: 6.4 KB


In [9]:
df_col['E_ESCV'] = (df_col['E_ESC2V'] +df_col['E_ESC2V'])/2
df_col['E_PROFV'] = (df_col['E_PROF1V'] +df_col['E_PROF1V'])/2
df_col['E_FAMV'] = (df_col['E_FAM1V'] +df_col['E_FAM1V'])/2
df_col['E_COMV'] = (df_col['E_COM1V'] +df_col['E_COM2V']+df_col['E_COM3V'])/3
df_col['E_ESTV'] = (df_col['E_EST1V'] +df_col['E_EST2V']+df_col['E_EST3V'])/3


In [10]:
df_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_aluno  62 non-null     int64  
 1   E_ESC1V   62 non-null     float64
 2   E_ESC2V   62 non-null     float64
 3   E_PROF1V  62 non-null     float64
 4   E_PROF2V  62 non-null     float64
 5   E_FAM1V   62 non-null     float64
 6   E_FAM2V   62 non-null     float64
 7   E_COM1V   62 non-null     float64
 8   E_COM2V   62 non-null     float64
 9   E_COM3V   62 non-null     float64
 10  E_EST1V   62 non-null     float64
 11  E_EST2V   62 non-null     float64
 12  E_EST3V   62 non-null     float64
 13  E_ESCV    62 non-null     float64
 14  E_PROFV   62 non-null     float64
 15  E_FAMV    62 non-null     float64
 16  E_COMV    62 non-null     float64
 17  E_ESTV    62 non-null     float64
dtypes: float64(17), int64(1)
memory usage: 8.8 KB


In [11]:
aluno_df = df_col[df_col['id_aluno']==19175]
aluno_df['E_ESC1V'].values[0]

1.0

In [12]:
app = Dash(__name__)
#app = Flask(__name__)

DIV_STYLE = {
  "height": "900px",
  "display": "flex",
  "justify-content": "center",
  "align-items": "center",
}

CHART_BOX_STYLE = {
  "display": "flex",
  "justify-content": "center",
  "flex-wrap": "wrap",
  "align-items": "center",
}

CHART_STYLE = {
  "flex": "0 0 33.33%",
  "maxWidth": "33.33%",
}

app.layout = html.Div(
  [
    html.Div(
      [
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,3.65],"yellow":[3.65,5.33],"red":[5.33,7]}},
            value=aluno_df['E_ESC1V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E-ESC1',
            max=7,
            min=0,
          ),
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.33],"red":[3.33,7]}},
            value=aluno_df['E_ESC2V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E-ESC2',
            max=7,
            min=0,
          ),
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.33],"yellow":[2.33,4.33],"red":[4.33,7]}},
            value=aluno_df['E_PROF1V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_PROF1',
            max=7,
            min=0,
          ),   
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.33],"yellow":[2.33,4],"red":[4,7]}},
            value=aluno_df['E_PROF2V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_PROF2',
            max=7,
            min=0,
          ),
          
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,3],"yellow":[3,5],"red":[5,7]}},
            value=aluno_df['E_FAM1V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_FAM1',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
      
      html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,3.33],"yellow":[3.33,5],"red":[5,7]}},
            value=aluno_df['E_FAM2V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_FAM2',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
      html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.66],"red":[3.66,7]}},
            value=aluno_df['E_COM1V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_COM1',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
    
     html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.66],"red":[3.66,7]}},
            value=aluno_df['E_COM2V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_COM2',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
      html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.66],"red":[3.66,7]}},
            value=aluno_df['E_COM3V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_COM3',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
    html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,4],"red":[4,7]}},
            value=aluno_df['E_EST1V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_EST1',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
    
    html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.33],"yellow":[2.33,4],"red":[4,7]}},
            value=aluno_df['E_EST2V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_EST2',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
       html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.33],"red":[3.33,7]}},
            value=aluno_df['E_EST3V'].values[0],
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_EST3',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
      ],
      style=CHART_BOX_STYLE,
    )
  ],
  style=DIV_STYLE,  
)

if __name__ == '__main__':
    app.run()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/eltonss/anaconda3/lib/python3.9/site-packages/werkzeug/serving.py", line 911, in prepare_socket
    s.bind(server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/eltonss/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_91933/3684657725.py", line 188, in <cell line: 187>
    app.run()
  File "/home/eltonss/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1980, in run
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/home/eltonss/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1188, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/home/eltonss/anaconda3/lib/python3.9/site-packages/werkzeug/serving.py", line 1062, in run_

TypeError: object of type 'NoneType' has no len()